<a href="https://colab.research.google.com/github/kanzaki-coder/deeplearning/blob/master/weed_gradcam_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#20分類のclassを入力
#芽生えの場合は1育成済みなら2を後ろに追加
classes=["hakidamegiku","hakidamegiku","hosoaogeitou","hosoaogeitou","ichibi",
         "ichibi","inubie","inubie","kosendangusa","kosendangusa","mameasagao",
         "mameasagao","mehishiba","mehishiba","ohishiba","ohishiba","oinutade",
         "oinutade","shiroza","shiroza"]


In [0]:
#10分類のclassを入力
classes=["hakidamegiku","hosoaogeitou","ichibi","inubie","kosendangusa","mameasagao",
         "mehishiba","ohishiba","oinutade","shiroza"]

In [0]:
#7分類のclassを入力
classes=["kikuf","hosoaogeitou","ichibi","inef","mameasagao","oinutade","shiroza"]

In [0]:
from keras.preprocessing import image
from PIL import Image 
from keras.preprocessing import image as images

img_path = "/content/cluster01/shiroza/shiroza_IMG_1542_19.JPG"
#↑判定させたいpathを指定する

input_size=224
x=img_path
img = Image.open(img_path).convert('RGB') 
img = img.resize((input_size,input_size))
y = np.array(img, dtype=np.float32)
y = y / 255.
plt.imshow(y)
print("以下の画像を判定します")

In [0]:

x=np.expand_dims(x,axis=0)
pred = model.predict(x, batch_size=1, verbose=0)

#最大の確率のものを抜き出す
pred_label = np.argmax(pred)
print(pred_label)

#もし各可能性を行列で見たい場合
print(pred)



In [0]:
plt.imshow(y)
print("この画像は"+classes[pred_label]+"です")

In [0]:
# 好きな画像のパスを取得
img_path = "/content/cluster01/shiroza/shiroza_IMG_1542_19.JPG"

#ここからgradcam

from keras.preprocessing import image
from PIL import Image 
from keras.preprocessing import image as images

img=image.load_img(img_path,target_size=(input_size,input_size))

# データの前処理
img = image.load_img(img_path, target_size=(input_size,input_size))

# xは形状が(224,224,3)のfloat32型のnumpy配列
x = image.img_to_array(img)/255.0
# この配列サイズが(1,224,224, 3)のバッチに変換するために次元を追加
x = np.expand_dims(x, axis=0)


In [0]:
from keras import backend as K 
import matplotlib.pyplot as plt

# 予測ベクトル
output = my_model.output[:,np.argmax(preds[0])]

# 最後の層であるblock5_conv3の出力特徴マップ
last_conv_layer = my_model.get_layer(last_layer_name)

# block5_conv3の出力特徴マップでの「ある」クラス勾配
grads = K.gradients(output,last_conv_layer.output)[0]


# 各エントリは特定の特徴マップの平均強度
pooled_grads = K.mean(grads,axis=(0,1,2))

#出力マップの値にアクセスするための関数
iterate = K.function([my_model.input], [pooled_grads, last_conv_layer.output[0]])

# 2つの値をNumpy配列で取得
pooled_grads_value, conv_layer_output_value = iterate([x])

# 「ある」クラスに関する「このチャネルの重要度」を特徴マップ配列の各チャネルに掛ける
for i in range(pooled_grads_value.shape[0]):
    conv_layer_output_value[:,:,i] *= pooled_grads_value[i]

# 最終的な特徴マップのチャネルごとの平均値がクラスの活性化のヒートマップ
heatmap = np.mean(conv_layer_output_value,axis=-1 )

# ヒートマップの後処理
print(heatmap.shape)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)

In [0]:
import cv2

# cv2を使って画像読み込み
img = cv2.imread(img_path)
# 元画像と同じサイズになるようにヒートマップのサイズを変更
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
# ヒートマップをRGBに変換
heatmap = np.uint8(255 * heatmap)
# ヒートマップをもとの画像に適応
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4はヒートマップの強度係数(もうちょっと強くしてもいい)
superimporsed_img = heatmap * 0.4 + img
# 画像を保存
cv2.imwrite('保存したい名前.jpg',superimporsed_img)

In [0]:
img = image.load_img('保存したい名前.jpg', target_size=(input_size,input_size))
img_tensor = image.img_to_array(img)
img_tensor /= 255.
plt.imshow(img_tensor)
plt.show